In [21]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [22]:
# -*- coding: utf-8 -*-
import dataset
# import keys_union
import keys_keras
import numpy as np
import torch
import time
import os, sys
sys.path.insert(0, os.getcwd())
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import TensorBoard
from keras.utils import plot_model


In [36]:
characters = keys_keras.alphabet_union[:]
# characters = keys_keras.alphabet[:]
from model import get_model
nclass = len(characters) + 1
trainroot = '../data/lmdb/train_competition_gen_25_v2/'
valroot = '../data/lmdb/valid_competition_gen_20/'
# modelPath = '../pretrain-models/keras.hdf5'
modelPath = ''
modelPath = '/mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/gru_competition_003996_*10.h5'
workers = 4
imgH = 32
imgW = 256
keep_ratio = False
random_sample = False
batchSize = 32

testSize = 128
n_len = 20
loss = 100000
interval = 300
LEARNING_RATE = 0.001
Learning_decay_step = 5000
PERCEPTION = 0.9
EPOCH_NUMS = 100000
MODEL_PATH = '/mnt/wuwenhui/git_ocr_project/keras_crnn/save_model'

LOG_FILE = 'log.txt'
SUMMARY_PATH = './log/'
if not os.path.exists(MODEL_PATH):
    print('Creating save model path!!')
    os.makedirs(MODEL_PATH)
if not os.path.exists(SUMMARY_PATH):
    os.makedirs(SUMMARY_PATH)

model, basemodel = get_model(
    height=imgH, nclass=nclass, learning_rate=LEARNING_RATE)

config = tf.ConfigProto(intra_op_parallelism_threads=4)
config.gpu_options.per_process_gpu_memory_fraction = PERCEPTION
KTF.set_session(tf.Session(config=config))

# 加载预训练参数
if os.path.exists(modelPath):
    # basemodel.load_weights(modelPath)
    model.load_weights(modelPath,by_name=True)
    print('model loaded')

# plot_model(basemodel, to_file='basemodel.png')
# plot_model(model, to_file='model.png')


def one_hot(text, length=20, characters=characters):
    label = np.zeros(length)
    for i, char in enumerate(text):
        index = characters.find(char)
        if index == -1:
            index = characters.find(u' ')
        if i < length:
            label[i] = index
    return label


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32, None, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, None, 64) 640         the_input[0][0]                  
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 16, None, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 16, None, 128 73856       pool1[0][0]                      
__________________________________________________________________________________________________
pool2 (Max

In [37]:

# 导入数据
if random_sample:
    sampler = dataset.randomSequentialSampler(train_dataset, batchSize)
else:
    sampler = None
train_dataset = dataset.lmdbDataset(root=trainroot, target_transform=one_hot)
# print(len(train_dataset))

test_dataset = dataset.lmdbDataset(
    root=valroot,
    transform=dataset.resizeNormalize((imgW, imgH)),
    target_transform=one_hot)

# 生成训练用数据
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batchSize,
    shuffle=True,
#     sampler=sampler,
    num_workers=2,
    collate_fn=dataset.alignCollate(
        imgH=imgH, imgW=imgW, keep_ratio=keep_ratio))

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=testSize, 
    num_workers=2,
    shuffle=False, )



nSamples:89602
nSamples:4995


In [38]:
# iter_data = iter(train_loader)
# %time x,y = next(iter_data)
# # print(y)
# x = x.numpy()
# X = x.reshape((-1, imgH, imgW, 1))
# Y = np.array(y)
# Length = int(imgW / 4) - 2
# batch = X.shape[0]
# X_train, Y_train = [
#     X, Y, np.ones(batch) * Length,
#     np.ones(batch) * n_len], np.ones(batch)
# X_train[0].shape,X_train[1].shape,X_train[2].shape,X_train[3].shape

In [39]:
# %timeit x,y = next(iter_data)


In [ ]:
j = 0

print('Start training!!')
for i in range(EPOCH_NUMS):
    for X, Y in train_loader:
        start = time.time()
        X = X.numpy()
        X = X.reshape((-1, imgH, imgW, 1))
#         print('Y ',Y)
        Y = np.array(Y)
        Length = int(imgW / 4) - 2
        batch = X.shape[0]
        X_train, Y_train = [
            X, Y, np.ones(batch) * Length,
            np.ones(batch) * n_len
        ], np.ones(batch)
#         print(Y_train)
        model.train_on_batch(X_train, Y_train)
        j +=1 
        if j % interval == 0:
            times = time.time() - start
            currentLoss_train = model.evaluate(X_train, Y_train)
            crrentLoss = 0
            for X,Y in test_loader:
                X = X.numpy()
                X = X.reshape((-1, imgH, imgW, 1))
                Y = Y.numpy()
                Y = np.array(Y)
                batch = X.shape[0]
                X_val, Y_val = [
                    X, Y, np.ones(batch) * Length,
                    np.ones(batch) * n_len
                ], np.ones(batch)
                crrentLoss += model.test_on_batch(X_val, Y_val)
            print('Learning rate is: ', LEARNING_RATE)
            now_time = time.strftime('%Y/%m/%d-%H:%M:%S',
                                     time.localtime(time.time()))
            print('Time: [%s]--Step/Epoch/Total: [%d/%d/%d]' % (now_time, j, i,
                                                                EPOCH_NUMS))
            print('\tTraining Loss is: [{}]'.format(currentLoss_train))
            print('\tVal Loss is: [{}]'.format(crrentLoss))
            print('\tSpeed is: [{}] Samples/Secs'.format(interval / times))
            path = MODEL_PATH + '/gru_competition_%06d_10.h5'%(crrentLoss*10)
            with open(LOG_FILE, mode='a') as log_file:
                log_str = now_time + '----global_step:' + str(
                    j) + '----loss:' + str(loss) + '\n'
                log_file.writelines(log_str)
            log_file.close()
            print('\tWriting to the file: log.txt')
            if crrentLoss < loss:
                loss = crrentLoss
                print("\tSave model to disk: {}".format(path))
                model.save(path)  


Start training!!
32/32 [==============================] - 3s 84ms/step
Learning rate is:  0.001
Time: [2018/10/19-16:26:09]--Step/Epoch/Total: [300/0/100000]
	Training Loss is: [10.7122220993042]
	Val Loss is: [447.5621395111084]
	Speed is: [513.3680204940656] Samples/Secs
	Writing to the file: log.txt
	Save model to disk: /mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/gru_competition_004475_10.h5
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-16:29:35]--Step/Epoch/Total: [600/0/100000]
	Training Loss is: [11.909153938293457]
	Val Loss is: [442.4048767089844]
	Speed is: [555.2241651061036] Samples/Secs
	Writing to the file: log.txt
	Save model to disk: /mnt/wuwenhui/git_ocr_project/keras_crnn/save_model/gru_competition_004424_10.h5
32/32 [==============================] - 0s 8ms/step
Learning rate is:  0.001
Time: [2018/10/19-16:33:02]--Step/Epoch/Total: [900/0/100000]
	Training Loss is: [5.38581657409668]
	Val Loss is: [445.0343022

32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-17:49:37]--Step/Epoch/Total: [8100/2/100000]
	Training Loss is: [8.156438827514648]
	Val Loss is: [422.2233781814575]
	Speed is: [522.9392525112314] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-17:52:50]--Step/Epoch/Total: [8400/2/100000]
	Training Loss is: [3.7790558338165283]
	Val Loss is: [422.7258734703064]
	Speed is: [562.4839407320254] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-17:56:01]--Step/Epoch/Total: [8700/3/100000]
	Training Loss is: [8.041082382202148]
	Val Loss is: [433.40369033813477]
	Speed is: [540.7806842429856] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/19-17:59:12]--Step/Epoch/Total: [9000/3/1

32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-19:18:59]--Step/Epoch/Total: [16500/5/100000]
	Training Loss is: [4.706073760986328]
	Val Loss is: [426.3087351322174]
	Speed is: [568.6914265879902] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-19:22:09]--Step/Epoch/Total: [16800/5/100000]
	Training Loss is: [2.0403428077697754]
	Val Loss is: [419.0067410469055]
	Speed is: [577.0977325949904] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-19:25:19]--Step/Epoch/Total: [17100/6/100000]
	Training Loss is: [1.9260475635528564]
	Val Loss is: [426.7507824897766]
	Speed is: [576.8146662217023] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-19:28:29]--Step/Epoch/Total: [17400

32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-20:47:49]--Step/Epoch/Total: [24900/8/100000]
	Training Loss is: [4.4883928298950195]
	Val Loss is: [418.7883152961731]
	Speed is: [509.2927245469234] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-20:50:59]--Step/Epoch/Total: [25200/8/100000]
	Training Loss is: [5.1897993087768555]
	Val Loss is: [423.52528619766235]
	Speed is: [547.1342755953274] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/19-20:54:10]--Step/Epoch/Total: [25500/9/100000]
	Training Loss is: [5.893773555755615]
	Val Loss is: [429.86864137649536]
	Speed is: [519.9683628258244] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-20:57:20]--Step/Epoch/Total: [258

32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-22:19:48]--Step/Epoch/Total: [33600/11/100000]
	Training Loss is: [2.9782323837280273]
	Val Loss is: [417.7498526573181]
	Speed is: [596.6708917122246] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/19-22:22:59]--Step/Epoch/Total: [33900/12/100000]
	Training Loss is: [3.7518529891967773]
	Val Loss is: [422.0793614387512]
	Speed is: [504.3787015454547] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/19-22:26:11]--Step/Epoch/Total: [34200/12/100000]
	Training Loss is: [4.581119537353516]
	Val Loss is: [430.6162109375]
	Speed is: [536.8214385362245] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/19-22:29:22]--Step/Epoch/Total: [34500

...

32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-10:33:28]--Step/Epoch/Total: [239100/85/100000]
	Training Loss is: [0.7332049012184143]
	Val Loss is: [539.8634848594666]
	Speed is: [518.5842217905808] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-10:36:38]--Step/Epoch/Total: [239400/85/100000]
	Training Loss is: [0.43259239196777344]
	Val Loss is: [548.2360153198242]
	Speed is: [531.4876647790127] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-10:39:49]--Step/Epoch/Total: [239700/85/100000]
	Training Loss is: [0.9653185606002808]
	Val Loss is: [551.0761580467224]
	Speed is: [559.387217092482] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-10:42:59]--Step/Epoch/Total:

32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-12:05:37]--Step/Epoch/Total: [247800/88/100000]
	Training Loss is: [0.3135406970977783]
	Val Loss is: [551.9345984458923]
	Speed is: [542.0699263463853] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-12:08:47]--Step/Epoch/Total: [248100/88/100000]
	Training Loss is: [0.1823655068874359]
	Val Loss is: [559.6252374649048]
	Speed is: [527.6469123025661] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/21-12:11:58]--Step/Epoch/Total: [248400/88/100000]
	Training Loss is: [0.4217061400413513]
	Val Loss is: [549.1121482849121]
	Speed is: [493.32526213111106] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-12:15:08]--Step/Epoch/Total

Learning rate is:  0.001
Time: [2018/10/21-13:34:32]--Step/Epoch/Total: [256200/91/100000]
	Training Loss is: [0.7635940313339233]
	Val Loss is: [558.4546751976013]
	Speed is: [540.248284958383] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/21-13:37:43]--Step/Epoch/Total: [256500/91/100000]
	Training Loss is: [0.3154665529727936]
	Val Loss is: [550.5613994598389]
	Speed is: [568.7119891816059] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 7ms/step
Learning rate is:  0.001
Time: [2018/10/21-13:40:52]--Step/Epoch/Total: [256800/91/100000]
	Training Loss is: [0.3504471778869629]
	Val Loss is: [546.1936101913452]
	Speed is: [505.6794187881131] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-13:44:04]--Step/Epoch/Total: [257100/91/100000]
	Training Loss is: [0.278263211250

32/32 [==============================] - 0s 5ms/step
Learning rate is:  0.001
Time: [2018/10/21-15:06:41]--Step/Epoch/Total: [264900/94/100000]
	Training Loss is: [0.37799757719039917]
	Val Loss is: [557.4275584220886]
	Speed is: [472.87665322900386] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-15:09:52]--Step/Epoch/Total: [265200/94/100000]
	Training Loss is: [0.43678537011146545]
	Val Loss is: [547.3119812011719]
	Speed is: [468.6631516374087] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-15:13:03]--Step/Epoch/Total: [265500/94/100000]
	Training Loss is: [0.7873505353927612]
	Val Loss is: [564.7476754188538]
	Speed is: [524.9463702393418] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 8ms/step
Learning rate is:  0.001
Time: [2018/10/21-15:16:15]--Step/Epoch/Tot

Learning rate is:  0.001
Time: [2018/10/21-16:35:34]--Step/Epoch/Total: [273300/97/100000]
	Training Loss is: [0.6433301568031311]
	Val Loss is: [557.5859384536743]
	Speed is: [525.5919059199364] Samples/Secs
	Writing to the file: log.txt
32/32 [==============================] - 0s 6ms/step
Learning rate is:  0.001
Time: [2018/10/21-16:38:45]--Step/Epoch/Total: [273600/97/100000]
	Training Loss is: [0.28910595178604126]
	Val Loss is: [564.3264317512512]
	Speed is: [567.6521287216482] Samples/Secs
	Writing to the file: log.txt


In [ ]:
x,y = iter(train_loader)